# Práctica Día 1 - Scraper Dinámico

## Finalidad

Este scraper dinámico está diseñado para extraer contenido de páginas web avanzadas que utilizan tecnologías como JavaScript para generar dinámicamente su contenido. A diferencia de los scrapers tradicionales que solo procesan HTML estático, este enfoque utiliza Selenium para controlar un navegador web real, permitiendo cargar y extraer información de sitios que requieren interacción con scripts o verificaciones adicionales.

### Celda 1: Importación de dependencias base
**Descripción:** Importa módulos esenciales para el manejo de dependencias, instalación de librerías y control de procesos.

In [1]:
import sys
import subprocess
import importlib.metadata
import time

### Celda 2: Lista de librerías necesarias
**Descripción:** Define una lista de las librerías requeridas para ejecutar el proyecto de scraping dinámico.

In [2]:
# Lista de librerías necesarias
required_libraries = [
    "selenium",
    "beautifulsoup4",
    "python-dotenv",
    "ollama",
    "ipython"
]

### Celda 3: Función para instalar librerías
**Descripción:** Verifica si las librerías necesarias están instaladas y las instala si es necesario.

In [3]:
# Función para comprobar e instalar librerías
def install_libraries(libraries):
    print("🔄 Comprobando e instalando librerías necesarias...\n")
    
    for i, lib in enumerate(libraries, 1):
        try:
            # Comprobar si la librería ya está instalada
            importlib.metadata.version(lib)
            print(f"✅ ({i}/{len(libraries)}) La librería '{lib}' ya está instalada. Se omite.\n")
        except importlib.metadata.PackageNotFoundError:
            # Instalar la librería si no está instalada
            try:
                print(f"👉 ({i}/{len(libraries)}) Instalando '{lib}'...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", lib], stdout=subprocess.DEVNULL)
                print(f"✅ '{lib}' instalada correctamente.\n")
            except Exception as e:
                print(f"⚠️ Error instalando la librería '{lib}': {e}\n")

    print("✨ Todas las librerías necesarias están instaladas. Puedes continuar ejecutando la siguiente celda.")

### Celda 4: Instalación de librerías
**Descripción:** Ejecuta la función para instalar todas las librerías requeridas y asegura que el entorno está configurado.

In [4]:
# Instalar librerías con mensajes
install_libraries(required_libraries)

🔄 Comprobando e instalando librerías necesarias...

✅ (1/5) La librería 'selenium' ya está instalada. Se omite.

✅ (2/5) La librería 'beautifulsoup4' ya está instalada. Se omite.

✅ (3/5) La librería 'python-dotenv' ya está instalada. Se omite.

✅ (4/5) La librería 'ollama' ya está instalada. Se omite.

✅ (5/5) La librería 'ipython' ya está instalada. Se omite.

✨ Todas las librerías necesarias están instaladas. Puedes continuar ejecutando la siguiente celda.


### Celda 5: Importación de todas las dependencias
**Descripción:** Importa todas las dependencias necesarias para el funcionamiento del scraper, incluyendo Selenium, BeautifulSoup y OpenAI.

In [5]:
# Importación de dependencias necesarias
import sys
import subprocess
import importlib.metadata
import os
import requests
import ollama
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
import time

### Celda 6: Declaramos las constantes
**Descripción:** Carga las constantes necesarias para trabajar con Ollama

In [6]:
# Cargamos las constantes
OLLAMA_API = "http://localhost:11434/api/chat"
MODEL = "llama3.2"

### Celda 6: Clase `DynamicWebsiteScraper`
**Descripción:** Implementa una clase para manejar el navegador, cargar páginas dinámicas y extraer contenido relevante.

In [7]:
# Clase que utiliza Selenium para cargar y extraer el contenido de páginas web dinámicas.
class DynamicWebsiteScraper:
    def __init__(self, driver_path):
        try:
            print("🔄 Creando instancia de DynamicWebsiteScraper...")
            options = Options()
            options.add_argument("--disable-gpu")
            options.add_argument("--no-sandbox")
            options.add_argument("--disable-dev-shm-usage")
            self.driver_service = Service(driver_path)
            self.driver = webdriver.Chrome(service=self.driver_service, options=options)
            print("✅ DynamicWebsiteScraper inicializado correctamente.\n")
        except Exception as e:
            print(f"❌ Error al inicializar DynamicWebsiteScraper: {e}")

    def scrape(self, url, wait_time=5):
        try:
            self.driver.get(url)
            time.sleep(wait_time)
            page_source = self.driver.page_source
            soup = BeautifulSoup(page_source, 'html.parser')
            title = soup.title.string if soup.title else "No tiene título"
            for tag in soup(["script", "style", "img", "input"]):
                tag.decompose()
            content = soup.get_text(separator="\n", strip=True)
            return {"title": title, "content": content}
        except Exception as e:
            print(f"❌ Error durante el scraping: {e}")
            return {"title": "Error", "content": ""}

    def close(self):
        try:
            self.driver.quit()
            print("✅ Navegador cerrado correctamente.\n")
        except Exception as e:
            print(f"❌ Error al cerrar el navegador: {e}")

### Celda 7: Función `user_prompt_for`
**Descripción:** Genera el mensaje de usuario basado en el título y contenido del sitio web extraído, que se utiliza como entrada para la API de Ollama.

In [8]:
# Genera el mensaje de usuario para el modelo Ollmaa.
def user_prompt_for(website):
    try:
        print("🔄 Generando el mensaje de usuario para el modelo GPT...")
        user_prompt = f"Estás viendo un sitio web titulado {website['title']}\n"
        user_prompt += (
            "El contenido de este sitio web es el siguiente:\n\n"
            f"{website['content']}\n\n"
            "Por favor, proporciona un breve resumen en español de este sitio web en formato Markdown."
        )
        print("✅ Mensaje de usuario generado correctamente.\n")
        return user_prompt
    except Exception as e:
        print(f"❌ Error al generar el mensaje de usuario: {e}")
        return ""

### Celda 8: Función `messages_for`
**Descripción:** Genera los mensajes requeridos para la API de Ollama

In [9]:
# Genera los mensajes requeridos para la API de OpenAI.
def messages_for(website):
    try:
        print("🔄 Generando los mensajes para la API de Ollama...")
        messages = [
            {"role": "user", "content": user_prompt_for(website)}
        ]
        print("✅ Mensajes generados correctamente para la API de Ollama.\n")
        return messages
    except Exception as e:
        print(f"❌ Error al generar los mensajes: {e}")
        return []

### Celda 9: Función `summarize`
**Descripción:** Realiza el scraping del sitio web, genera mensajes y solicita un resumen utilizando la API de Ollama.

In [16]:
# Función para generar un resumen
def summarize(url, scraper):
    """
    Realiza el scraping del sitio web y utiliza la API de Ollama para generar un resumen.
    :param url: URL del sitio web
    :param scraper: Instancia de DynamicWebsiteScraper
    :return: Resumen en Markdown
    """
    try:
        print(f"🔄 Realizando scraping en la URL: {url}")
        website = scraper.scrape(url)  # Usamos el scraper para obtener el contenido dinámico
        print(f"✅ Scraping completado con éxito en la URL: {url}\n")

        print("🔄 Generando el resumen a través de la API de OpenAI...")
        response = ollama.chat(model=MODEL, messages=messages_for(website))
        
        # Ajusta el acceso al contenido según la respuesta observada
        if hasattr(response, "message") and hasattr(response.message, "content"):
            return response.message.content
        else:
            raise ValueError("Formato de respuesta inesperado.")

    except Exception as e:
        print(f"❌ Error al generar el resumen: {e}")
        return "Error al generar el resumen."

### Celda 10: Función `display_summary`
**Descripción:** Llama a la función `summarize`, obtiene el resumen del sitio web y lo muestra en formato Markdown.

In [11]:
# Genera y muestra el resumen de un sitio web en formato Markdown utilizando un scraper.
def display_summary(url, scraper):
    try:
        print(f"🔄 Iniciando el proceso para generar el resumen del sitio web: {url}")
        summary = summarize(url, scraper)  # Llamada a summarize con url y scraper
        
        if summary.startswith("Error"):
            print(f"❌ No se pudo generar el resumen para: {url}")
        else:
            print(f"✅ Resumen generado correctamente para: {url}\n")
            display(Markdown(summary))  # Mostrar el resumen en Markdown
    except Exception as e:
        print(f"❌ Ocurrió un error durante la visualización del resumen: {e}")

## Configuración inicial para el Scraper Dinámico

Para que el scraper dinámico funcione correctamente, es necesario configurar **ChromeDriver**, un controlador que permite a Selenium interactuar con el navegador Google Chrome. En esta sección, aprenderás a descargar, configurar y agregar ChromeDriver al PATH del sistema para garantizar la compatibilidad y el correcto funcionamiento del scraper. Sigue los pasos indicados para preparar tu entorno de trabajo.

## 1. Descarga de ChromeDriver

1. Ve a la página oficial de ChromeDriver:  
   [https://chromedriver.chromium.org/downloads](https://chromedriver.chromium.org/downloads)

2. Descarga la versión de **ChromeDriver** que sea compatible con la versión de tu navegador Google Chrome:
   - **En Google Chrome**, verifica tu versión desde:
     - `Configuración > Acerca de Google Chrome`.

3. Una vez descargado el archivo, extrae el contenido en una carpeta de tu sistema. Algunos ejemplos de ubicaciones comunes:
   - En **Windows**: `C:\chromedriver\chromedriver.exe`
   - En **Linux/Mac**: `/usr/local/bin/chromedriver`

---

⚠️ **Nota importante**: Asegúrate de que la versión de ChromeDriver coincida con tu navegador Google Chrome, de lo contrario, podrían ocurrir errores de compatibilidad.

## 2. Configuración en el código

Para que tu script pueda encontrar el ejecutable de ChromeDriver, debes establecer correctamente su **ruta**. Aquí tienes algunos ejemplos dependiendo de tu sistema operativo:

```python
# En Windows
CHROME_DRIVER_PATH = "C:\\chromedriver\\chromedriver.exe"

# En Linux o Mac
CHROME_DRIVER_PATH = "/usr/local/bin/chromedriver"


## 3. Agregar ChromeDriver al PATH (opcional)

Si prefieres no especificar la ruta completa cada vez, puedes agregar **ChromeDriver** al **PATH** de tu sistema operativo. Esto hará que sea accesible globalmente desde cualquier script.

### En Windows:
1. Abre `Configuración > Sistema > Acerca de > Configuración avanzada del sistema`.
2. Haz clic en el botón **Variables de entorno**.
3. Selecciona la variable `PATH` y haz clic en **Editar**.
4. Añade la ruta donde se encuentra `chromedriver.exe`, por ejemplo:  
   `C:\chromedriver`.

---

### En Linux/Mac:
1. Abre el archivo de configuración de tu terminal (`.bashrc` o `.zshrc`).
2. Añade la siguiente línea al final del archivo:  
   ```bash
   export PATH=$PATH:/usr/local/bin
3. Guarda el archivo y recarga el terminal ejecutando:
4. ````bash
   source ~/.bashrc  # o source ~/.zshrc


## 4. Conclusión

Con ChromeDriver correctamente configurado y accesible, puedes usarlo en tus proyectos de **web scraping** con Selenium. Si sigues teniendo problemas, revisa que:
- La versión de ChromeDriver coincida con la versión de tu navegador Google Chrome.
- La ruta al ejecutable esté correctamente definida.

¡Ahora estás listo para continuar con el código de scraping!

### Celda 15: Ejecución del scraper
**Descripción:** Configura el scraper con el camino al ChromeDriver, realiza el scraping del sitio web y genera un resumen, cerrando el navegador al finalizar.

In [17]:
CHROME_DRIVER_PATH = "C:\\tmp\\chromedriver-win64\\chromedriver.exe"  # Ruta a tu ChromeDriver

try:
    print("🔄 Iniciando la configuración del scraper...")
    scraper = DynamicWebsiteScraper(CHROME_DRIVER_PATH)
    print("✅ Scraper configurado correctamente.\n")
    
    url = "https://openai.com"
    print(f"🔄 Generando resumen para el sitio web: {url}...\n")
    display_summary(url, scraper)

except Exception as e:
    print(f"❌ Ocurrió un error durante el proceso: {e}")
    
finally:
    try:
        print("🔄 Cerrando el scraper...")
        scraper.close()
        print("✅ Scraper cerrado correctamente.")
    except Exception as close_error:
        print(f"❌ Error al cerrar el scraper: {close_error}")


🔄 Iniciando la configuración del scraper...
🔄 Creando instancia de DynamicWebsiteScraper...
✅ DynamicWebsiteScraper inicializado correctamente.

✅ Scraper configurado correctamente.

🔄 Generando resumen para el sitio web: https://openai.com...

🔄 Iniciando el proceso para generar el resumen del sitio web: https://openai.com
🔄 Realizando scraping en la URL: https://openai.com
✅ Scraping completado con éxito en la URL: https://openai.com

🔄 Generando el resumen a través de la API de OpenAI...
🔄 Generando los mensajes para la API de Ollama...
🔄 Generando el mensaje de usuario para el modelo GPT...
✅ Mensaje de usuario generado correctamente.

✅ Mensajes generados correctamente para la API de Ollama.

✅ Resumen generado correctamente para: https://openai.com



**Resumen del sitio web ChatGPT**

El sitio web de ChatGPT es una plataforma de inteligencia artificial que ofrece conversación y respuesta a preguntas con inteligencia artificial. Ofrece varias versiones de chatbots, incluyendo:

*   GPT-4
*   Sora
*   DALL·E 3

El sitio web también proporciona una API para desarrolladores, así como un plan de precios y documentación.
Además, ofrece recursos adicionales, como artículos de investigación sobre la inteligencia artificial y políticas de privacidad.

El sitio web es de propiedad de OpenAI y se enfoca en proporcionar herramientas de inteligencia artificial para empresas, equipos y individuos.

🔄 Cerrando el scraper...
✅ Navegador cerrado correctamente.

✅ Scraper cerrado correctamente.
